In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !pip install torchmetrics
# !git clone https://github.com/Hpeox/Intellectual_manifest.git

In [4]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
# from tqdm import tqdm

import cv2, time
# from PIL import Image
# from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchmetrics
import torchvision.models as models
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

import albumentations as A


# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [5]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [6]:
# 读取数据集
train_path = glob.glob("./data/sample/*/*")
test_path = glob.glob("./data/test/*")

train_path.sort()
test_path.sort()

# train_df = pd.read_csv("data/train.csv")
# train_df = train_df.sort_values(by="name")
# train_label = train_df["label"].values

train_label = [np.int64(0) for i in range(480)]+[np.int64(1) for i in range(2400)]

# 自定义数据集
# 带有图片缓存的逻辑
DATA_CACHE = {}


class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        if self.img_path[index] in DATA_CACHE:
            img = DATA_CACHE[self.img_path[index]]
        else:
            img = cv2.imread(self.img_path[index])
            DATA_CACHE[self.img_path[index]] = img
        if self.transform is not None:
            img = self.transform(image=img)["image"]
        img = img.transpose([2, 0, 1])
        return img, torch.from_numpy(np.array(self.img_label[index]))

    def __len__(self):
        return len(self.img_path)

dataset = XunFeiDataset(
    train_path,
    train_label,
    A.Compose(
        [
            A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    ),
)
train_dataset, val_dataset = torch.utils.data.random_split(
    dataset=dataset, lengths=[0.9, 0.1], generator=torch.Generator().manual_seed(42)
)

# 训练集
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)

# 验证集
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=False
)

# 测试集
test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(
        test_path,
        [0] * len(test_path),
        A.Compose(
            [
                A.Resize(256, 256),
                A.RandomCrop(224, 224),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.5),
                A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
            ]
        ),
    ),
    batch_size=50,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

In [7]:
class XunFeiNet50(nn.Module):
    def __init__(self):
        super(XunFeiNet50, self).__init__()
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet101(nn.Module):
    def __init__(self):
        super(XunFeiNet101, self).__init__()
        model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet152(nn.Module):
    def __init__(self):
        super(XunFeiNet152, self).__init__()
        model = models.resnet152(weights=models.ResNet152_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(2048, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet34(nn.Module):
    def __init__(self):
        super(XunFeiNet34, self).__init__()
        model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet18(nn.Module):
    def __init__(self):
        super(XunFeiNet18, self).__init__()
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

class XunFeiNet121(nn.Module):
    def __init__(self):
        super(XunFeiNet121, self).__init__()
        model = models.densenet121(num_classes=2)
        self.resnet = model
    def forward(self, img):
        out = self.resnet(img)
        return out

model121 = XunFeiNet121().to(device)
model18 = XunFeiNet18().to(device)
model34 = XunFeiNet34().to(device)
model50 = XunFeiNet50().to(device)
model101 = XunFeiNet101().to(device)
model152 = XunFeiNet152().to(device)
# model = model.to(device)
criterion = nn.CrossEntropyLoss().cuda()

In [8]:
# 模型训练
def train(train_loader, model, criterion, optimizer):
    start = time.time()
    # start_batch = [start, 0]
    model.train()
    train_loss = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    for _, (input, target) in enumerate(train_loader):
        input = input.to(device)
        target = target.to(device)
        output = model(input)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if (i + 1) % 30 == 0:
        #     start_batch[(1+(i + 1) // 100) % 2] = time.time()
        #     print(
        #         "Train loss",
        #         loss.item(),
        #         "t={}s".format(
        #             start_batch[(1+(i + 1) // 100) % 2]
        #             - start_batch[((i + 1) // 100) % 2]
        #         ),
        #     )

        preds = torch.cat((preds, output.cpu().argmax(1)))
        target_all = torch.cat((target_all, target.cpu()))

        train_loss += loss.item()

    val_acc = torchmetrics.functional.classification.multiclass_f1_score(
        preds, target_all, num_classes=2, average="macro"
    )
    print("t={}s".format(time.time() - start))
    print("F1 score", val_acc)
    return train_loss / len(train_loader)


# 模型验证
def validate(val_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    preds = torch.tensor([])
    target_all = torch.tensor([])
    val_loss=0.0
    with torch.no_grad():
        for _, (input, target) in enumerate(val_loader):
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)
            # val_acc += (output.argmax(1) == target).sum().item()
            preds = torch.cat((preds, output.cpu().argmax(1)))
            target_all = torch.cat((target_all, target.cpu()))

        val_acc = torchmetrics.functional.classification.multiclass_f1_score(
            preds, target_all, num_classes=2, average="macro"
        )
        val_loss += loss.item()
    # return val_acc / len(val_loader.dataset)
    return val_acc,val_loss/len(val_loader)


# 模型预测
def predict(test_loader, model):
    model.eval()

    test_pred = []
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input = input.to(device)
            output = model(input)
            test_pred.append(output.data.cpu().numpy())

    return np.vstack(test_pred)

In [ ]:
epochs = 50
print("========resnet18========")
model=model18
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss, "\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_18.pth")
print("========resnet34========")
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_34.pth")

print("========resnet50========")
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_50.pth")

print("========resnet101========")
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_101.pth")

print("========resnet152========")
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_152.pth")

print("========densnet121========")
model=model121
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_121.pth")

========resnet18========
t=24.906310319900513s
F1 score tensor(0.6211)
epoch 1 : train_loss: 0.541062899479052 val_loss: 0.08855052590370179 
val_f1: tensor(0.7541) 

t=10.155979633331299s
F1 score tensor(0.7262)
epoch 2 : train_loss: 0.4600911402120823 val_loss: 0.08177627921104431 
val_f1: tensor(0.7360) 

t=10.204305410385132s
F1 score tensor(0.7476)
epoch 3 : train_loss: 0.40938178286319826 val_loss: 0.06173466444015503 
val_f1: tensor(0.8046) 

t=10.660547018051147s
F1 score tensor(0.7851)
epoch 4 : train_loss: 0.36282722324859806 val_loss: 0.05929139256477356 
val_f1: tensor(0.8402) 

t=11.069799184799194s
F1 score tensor(0.7950)
epoch 5 : train_loss: 0.3364242286216922 val_loss: 0.04950132369995117 
val_f1: tensor(0.8189) 

t=11.352064847946167s
F1 score tensor(0.8193)
epoch 6 : train_loss: 0.2989829767767976 val_loss: 0.05007819533348083 
val_f1: tensor(0.8815) 

t=10.538376331329346s
F1 score tensor(0.8249)
epoch 7 : train_loss: 0.28079985663658236 val_loss: 0.0510823726654052

In [9]:
epochs = 50
# print("========resnet18========")
# model=model18
# model.load_state_dict(torch.load("./model/baseline25_18.pth"))
# optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
# for epoch in range(epochs):
#     train_loss = train(train_loader, model, criterion, optimizer)
#     val_acc,val_loss = validate(val_loader, model, criterion)
#     print(
#         "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss, "\nval_f1:", val_acc, "\n"
#     )
# torch.save(model.state_dict(),"./model/baseline25_18.pth")
# print("========resnet34========")
# model=model34
# optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
# model.load_state_dict(torch.load("./model/baseline25_34.pth"))
# for epoch in range(epochs):
#     train_loss = train(train_loader, model, criterion, optimizer)
#     val_acc,val_loss = validate(val_loader, model, criterion)
#     print(
#         "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
#     )
# torch.save(model.state_dict(),"./model/baseline25_34.pth")

print("========resnet50========")
f1_50=[]
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_50.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_50.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_50.pth")

print("========resnet101========")
f1_101=[]
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_101.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_101.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_101.pth")

print("========resnet152========")
model=model152
f1_152=[]
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_152.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_152.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_152.pth")

print("========densnet121========")
f1_121=[]
model=model121
optimizer = torch.optim.AdamW(model.parameters(), 0.000005,weight_decay=5e-2)
model.load_state_dict(torch.load("./model/baseline25_121.pth"))
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    f1_121.append(val_acc)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"\nval_f1:", val_acc, "\n"
    )
torch.save(model.state_dict(),"./model/baseline25_121.pth")

========resnet50========
t=18.050381898880005s
F1 score tensor(0.9495)
epoch 1 : train_loss: 0.0691385610535005 val_loss: 0.00710305737124549 
val_f1: tensor(0.9329) 

t=119.51899600028992s
F1 score tensor(0.9540)
epoch 2 : train_loss: 0.0672593042369426 val_loss: 0.005375111268626319 
val_f1: tensor(0.9307) 

t=13.315320253372192s
F1 score tensor(0.9534)
epoch 3 : train_loss: 0.06983617539115158 val_loss: 0.012161867486106025 
val_f1: tensor(0.9442) 

t=84.046071767807s
F1 score tensor(0.9560)
epoch 4 : train_loss: 0.06861240846783291 val_loss: 0.0017942395061254501 
val_f1: tensor(0.9433) 

t=13.68542218208313s
F1 score tensor(0.9623)
epoch 5 : train_loss: 0.057293530760539904 val_loss: 0.0019546347773737377 
val_f1: tensor(0.9433) 

t=13.439164638519287s
F1 score tensor(0.9540)
epoch 6 : train_loss: 0.06320193257573763 val_loss: 0.0031554334693484837 
val_f1: tensor(0.9442) 

t=13.498860597610474s
F1 score tensor(0.9619)
epoch 7 : train_loss: 0.06164775211593987 val_loss: 0.01051984

In [14]:
f1_152

[tensor(0.9125),
 tensor(0.9285),
 tensor(0.9262),
 tensor(0.9220),
 tensor(0.9125),
 tensor(0.9385),
 tensor(0.9085),
 tensor(0.9285),
 tensor(0.9085),
 tensor(0.9262),
 tensor(0.9433),
 tensor(0.9508),
 tensor(0.9250),
 tensor(0.9364),
 tensor(0.8983),
 tensor(0.9375),
 tensor(0.9523),
 tensor(0.9237),
 tensor(0.9250),
 tensor(0.9516),
 tensor(0.9442),
 tensor(0.9508),
 tensor(0.9307),
 tensor(0.9451),
 tensor(0.9250),
 tensor(0.9375),
 tensor(0.9307),
 tensor(0.9566),
 tensor(0.9340),
 tensor(0.9385),
 tensor(0.9508),
 tensor(0.9375),
 tensor(0.9580),
 tensor(0.9573),
 tensor(0.9643),
 tensor(0.9274),
 tensor(0.9508),
 tensor(0.9580),
 tensor(0.9237),
 tensor(0.9329),
 tensor(0.9442),
 tensor(0.9442),
 tensor(0.9559),
 tensor(0.9395),
 tensor(0.9508),
 tensor(0.9500),
 tensor(0.9637),
 tensor(0.9631),
 tensor(0.9573),
 tensor(0.9643)]

101/152/121 100epochs f1_marco~0.96

========resnet50========

t=18.050381898880005s
F1 score tensor(0.9495)
epoch 1 : train_loss: 0.0691385610535005 val_loss: 0.00710305737124549 
val_f1: tensor(0.9329) 

t=119.51899600028992s
F1 score tensor(0.9540)
epoch 2 : train_loss: 0.0672593042369426 val_loss: 0.005375111268626319 
val_f1: tensor(0.9307) 

t=13.315320253372192s
F1 score tensor(0.9534)
epoch 3 : train_loss: 0.06983617539115158 val_loss: 0.012161867486106025 
val_f1: tensor(0.9442) 

t=84.046071767807s
F1 score tensor(0.9560)
epoch 4 : train_loss: 0.06861240846783291 val_loss: 0.0017942395061254501 
val_f1: tensor(0.9433) 

t=13.68542218208313s
F1 score tensor(0.9623)
epoch 5 : train_loss: 0.057293530760539904 val_loss: 0.0019546347773737377 
val_f1: tensor(0.9433) 

t=13.439164638519287s
F1 score tensor(0.9540)
epoch 6 : train_loss: 0.06320193257573763 val_loss: 0.0031554334693484837 
val_f1: tensor(0.9442) 

t=13.498860597610474s
F1 score tensor(0.9619)
epoch 7 : train_loss: 0.06164775211593987 val_loss: 0.010519845618142022 
val_f1: tensor(0.9274) 

t=13.78493618965149s
F1 score tensor(0.9645)
epoch 8 : train_loss: 0.05571419991161904 val_loss: 0.002365339547395706 
val_f1: tensor(0.9631) 

t=13.073767900466919s
F1 score tensor(0.9598)
epoch 9 : train_loss: 0.06034181280820458 val_loss: 0.010115183062023587 
val_f1: tensor(0.9340) 

t=12.936011791229248s
F1 score tensor(0.9682)
epoch 10 : train_loss: 0.05255766804502518 val_loss: 0.0044179898169305586 
val_f1: tensor(0.9700) 

t=13.358749866485596s
F1 score tensor(0.9682)
epoch 11 : train_loss: 0.053271972091385615 val_loss: 0.004408222105767991 
val_f1: tensor(0.9152) 

t=12.464449882507324s
F1 score tensor(0.9724)
epoch 12 : train_loss: 0.05010973413210408 val_loss: 0.0064985549284352195 
val_f1: tensor(0.9364) 

t=12.371459007263184s
F1 score tensor(0.9667)
epoch 13 : train_loss: 0.05006366570330696 val_loss: 0.0015366951831512982 
val_f1: tensor(0.9508) 

t=12.433188199996948s
F1 score tensor(0.9656)
epoch 14 : train_loss: 0.05201281249569154 val_loss: 0.0013746606806914012 
val_f1: tensor(0.9442) 

t=12.229204893112183s
F1 score tensor(0.9674)
epoch 15 : train_loss: 0.058269254863262177 val_loss: 0.002577404802044233 
val_f1: tensor(0.9516) 

t=12.321924209594727s
F1 score tensor(0.9725)
epoch 16 : train_loss: 0.047542552279745354 val_loss: 0.0010372667262951534 
val_f1: tensor(0.9573) 

t=13.919592142105103s
F1 score tensor(0.9689)
epoch 17 : train_loss: 0.048000054658149126 val_loss: 0.0016470426279637548 
val_f1: tensor(0.9375) 

t=13.235016822814941s
F1 score tensor(0.9639)
epoch 18 : train_loss: 0.05213661681584738 val_loss: 0.003468662914302614 
val_f1: tensor(0.9500) 

t=12.191822528839111s
F1 score tensor(0.9645)
epoch 19 : train_loss: 0.04558655132114151 val_loss: 0.003111734158462948 
val_f1: tensor(0.9451) 

t=12.19049334526062s
F1 score tensor(0.9616)
epoch 20 : train_loss: 0.05445467392385098 val_loss: 0.008555764125453101 
val_f1: tensor(0.9375) 

t=12.219496011734009s
F1 score tensor(0.9742)
epoch 21 : train_loss: 0.0438526945884259 val_loss: 0.0015162933203909132 
val_f1: tensor(0.9559) 

t=12.212045192718506s
F1 score tensor(0.9721)
epoch 22 : train_loss: 0.04292828495970663 val_loss: 0.002733039359251658 
val_f1: tensor(0.9631) 

t=12.71650767326355s
F1 score tensor(0.9746)
epoch 23 : train_loss: 0.04088526266845472 val_loss: 0.006953562299410502 
val_f1: tensor(0.9643) 

t=12.718422174453735s
F1 score tensor(0.9719)
epoch 24 : train_loss: 0.04532581422891882 val_loss: 0.0063143836127387155 
val_f1: tensor(0.9364) 

t=12.717062950134277s
F1 score tensor(0.9712)
epoch 25 : train_loss: 0.04962706294221183 val_loss: 0.002550632589393192 
val_f1: tensor(0.9580) 

t=12.816307544708252s
F1 score tensor(0.9831)
epoch 26 : train_loss: 0.03586396351916554 val_loss: 0.0035500344302919176 
val_f1: tensor(0.9516) 

t=13.262090682983398s
F1 score tensor(0.9667)
epoch 27 : train_loss: 0.04903863640617073 val_loss: 0.010203422771559821 
val_f1: tensor(0.9500) 

t=13.053552627563477s
F1 score tensor(0.9769)
epoch 28 : train_loss: 0.0398748473747185 val_loss: 0.014853997362984551 
val_f1: tensor(0.9580) 

t=13.088923215866089s
F1 score tensor(0.9768)
epoch 29 : train_loss: 0.038582911579812196 val_loss: 0.0008626203052699566 
val_f1: tensor(0.9500) 

t=13.10727310180664s
F1 score tensor(0.9795)
epoch 30 : train_loss: 0.033582822574923434 val_loss: 0.0018235310498211118 
val_f1: tensor(0.9375) 

t=13.080960512161255s
F1 score tensor(0.9781)
epoch 31 : train_loss: 0.03185533136876737 val_loss: 0.0002390436776396301 
val_f1: tensor(0.9695) 

t=13.443254709243774s
F1 score tensor(0.9704)
epoch 32 : train_loss: 0.04358885159003146 val_loss: 0.0071899981962309945 
val_f1: tensor(0.9690) 

t=13.272247552871704s
F1 score tensor(0.9811)
epoch 33 : train_loss: 0.03076519516158711 val_loss: 0.0003434082803626855 
val_f1: tensor(0.9631) 

t=13.401961088180542s
F1 score tensor(0.9748)
epoch 34 : train_loss: 0.03527479893297969 val_loss: 0.0036974619660112592 
val_f1: tensor(0.9559) 

t=12.298096895217896s
F1 score tensor(0.9754)
epoch 35 : train_loss: 0.037553080509957155 val_loss: 0.007435057726171281 
val_f1: tensor(0.9566) 

t=12.184111595153809s
F1 score tensor(0.9774)
epoch 36 : train_loss: 0.03290384988312 val_loss: 0.014869305822584365 
val_f1: tensor(0.9573) 

t=12.176159143447876s
F1 score tensor(0.9817)
epoch 37 : train_loss: 0.03495607398346894 val_loss: 0.010790221393108368 
val_f1: tensor(0.9508) 

t=12.230472087860107s
F1 score tensor(0.9804)
epoch 38 : train_loss: 0.032468566242720424 val_loss: 0.00037124163160721463 
val_f1: tensor(0.9500) 

t=12.208173274993896s
F1 score tensor(0.9817)
epoch 39 : train_loss: 0.031080882202971866 val_loss: 0.0024180099782016543 
val_f1: tensor(0.9643) 

t=12.225772380828857s
F1 score tensor(0.9790)
epoch 40 : train_loss: 0.03457977227222772 val_loss: 0.0006376121503611406 
val_f1: tensor(0.9516) 

t=12.315294027328491s
F1 score tensor(0.9867)
epoch 41 : train_loss: 0.025372688408458122 val_loss: 0.00018912465828988288 
val_f1: tensor(0.9451) 

t=12.202412605285645s
F1 score tensor(0.9817)
epoch 42 : train_loss: 0.031068563212663578 val_loss: 0.006581776258018281 
val_f1: tensor(0.9637) 

t=12.19942855834961s
F1 score tensor(0.9833)
epoch 43 : train_loss: 0.027180899794064003 val_loss: 0.0010216219557656182 
val_f1: tensor(0.9631) 

t=12.201426029205322s
F1 score tensor(0.9831)
epoch 44 : train_loss: 0.031000925210752973 val_loss: 0.000302206181610624 
val_f1: tensor(0.9631) 

t=12.199506044387817s
F1 score tensor(0.9852)
epoch 45 : train_loss: 0.025490490550083327 val_loss: 0.0062893446948793195 
val_f1: tensor(0.9573) 

t=12.185949802398682s
F1 score tensor(0.9812)
epoch 46 : train_loss: 0.030016719242902817 val_loss: 0.0012952562214599715 
val_f1: tensor(0.9695) 

t=12.21652340888977s
F1 score tensor(0.9853)
epoch 47 : train_loss: 0.025380882198841853 val_loss: 0.0004253626749333408 
val_f1: tensor(0.9573) 

t=12.22187089920044s
F1 score tensor(0.9902)
epoch 48 : train_loss: 0.02161938043161767 val_loss: 0.009519161449538337 
val_f1: tensor(0.9631) 

t=12.204710721969604s
F1 score tensor(0.9853)
epoch 49 : train_loss: 0.02642579221477111 val_loss: 0.00034511720554696187 
val_f1: tensor(0.9695) 

t=12.211538791656494s
F1 score tensor(0.9859)
epoch 50 : train_loss: 0.026623724889366623 val_loss: 0.012284044590261247 
val_f1: tensor(0.9460) 

========resnet101========

t=19.526291131973267s
F1 score tensor(0.9344)
epoch 1 : train_loss: 0.09247006551810989 val_loss: 0.007735187808672587 
val_f1: tensor(0.8963) 

t=19.680978059768677s
F1 score tensor(0.9399)
epoch 2 : train_loss: 0.09239812735698105 val_loss: 0.0039001951615015664 
val_f1: tensor(0.8841) 

t=19.57154870033264s
F1 score tensor(0.9404)
epoch 3 : train_loss: 0.08633305729125385 val_loss: 0.006387736234400008 
val_f1: tensor(0.9110) 

t=19.557839155197144s
F1 score tensor(0.9399)
epoch 4 : train_loss: 0.08453640206084576 val_loss: 0.00500808780392011 
val_f1: tensor(0.8891) 

t=19.56749725341797s
F1 score tensor(0.9441)
epoch 5 : train_loss: 0.08093954888344915 val_loss: 0.009119995766215853 
val_f1: tensor(0.9016) 

t=19.512648820877075s
F1 score tensor(0.9454)
epoch 6 : train_loss: 0.07947704984151104 val_loss: 0.01154826001988517 
val_f1: tensor(0.8802) 

t=19.481063842773438s
F1 score tensor(0.9531)
epoch 7 : train_loss: 0.07648419149587919 val_loss: 0.01589707202381558 
val_f1: tensor(0.9070) 

t=19.52741050720215s
F1 score tensor(0.9523)
epoch 8 : train_loss: 0.07283778401827555 val_loss: 0.004214384903510411 
val_f1: tensor(0.9000) 

t=19.543887853622437s
F1 score tensor(0.9503)
epoch 9 : train_loss: 0.07147511868611162 val_loss: 0.0033254375060399375 
val_f1: tensor(0.9095) 

t=19.510855197906494s
F1 score tensor(0.9483)
epoch 10 : train_loss: 0.07280757143679592 val_loss: 0.001131321820947859 
val_f1: tensor(0.9016) 

t=19.53829026222229s
F1 score tensor(0.9602)
epoch 11 : train_loss: 0.06903982752313216 val_loss: 0.002896857344441944 
val_f1: tensor(0.9070) 

t=19.54599142074585s
F1 score tensor(0.9525)
epoch 12 : train_loss: 0.06714530439794432 val_loss: 0.0008626553850869337 
val_f1: tensor(0.9194) 

t=19.612682342529297s
F1 score tensor(0.9553)
epoch 13 : train_loss: 0.06229389381284515 val_loss: 0.012168153292602964 
val_f1: tensor(0.9125) 

t=19.527552127838135s
F1 score tensor(0.9507)
epoch 14 : train_loss: 0.07358003799217167 val_loss: 0.005843109140793483 
val_f1: tensor(0.8974) 

t=19.526849031448364s
F1 score tensor(0.9661)
epoch 15 : train_loss: 0.058934454507205955 val_loss: 0.043760438760121666 
val_f1: tensor(0.9181) 

t=19.54566526412964s
F1 score tensor(0.9607)
epoch 16 : train_loss: 0.057652778555581596 val_loss: 0.009694207045767043 
val_f1: tensor(0.8946) 

t=19.571969509124756s
F1 score tensor(0.9657)
epoch 17 : train_loss: 0.053685034791950456 val_loss: 0.018062535259458754 
val_f1: tensor(0.9262) 

t=19.509676933288574s
F1 score tensor(0.9634)
epoch 18 : train_loss: 0.057847524883892436 val_loss: 0.0046685197287135655 
val_f1: tensor(0.9194) 

t=19.717072010040283s
F1 score tensor(0.9552)
epoch 19 : train_loss: 0.061336334357842984 val_loss: 0.013491056031650968 
val_f1: tensor(0.8946) 

t=19.54115915298462s
F1 score tensor(0.9756)
epoch 20 : train_loss: 0.04529440399535276 val_loss: 0.0018315896805789736 
val_f1: tensor(0.9375) 

t=19.54300546646118s
F1 score tensor(0.9620)
epoch 21 : train_loss: 0.05610583586917247 val_loss: 0.012857539786232842 
val_f1: tensor(0.9262) 

t=19.366201162338257s
F1 score tensor(0.9697)
epoch 22 : train_loss: 0.04883409399010333 val_loss: 0.0007128039788868693 
val_f1: tensor(0.9152) 

t=19.230911254882812s
F1 score tensor(0.9688)
epoch 23 : train_loss: 0.047462283788869776 val_loss: 0.004430219944980409 
val_f1: tensor(0.9070) 

t=19.229432821273804s
F1 score tensor(0.9624)
epoch 24 : train_loss: 0.05847034081931651 val_loss: 0.003769100540214115 
val_f1: tensor(0.9166) 

t=19.292271375656128s
F1 score tensor(0.9675)
epoch 25 : train_loss: 0.05201420820042215 val_loss: 0.0013468950572941038 
val_f1: tensor(0.9070) 

t=19.12525177001953s
F1 score tensor(0.9783)
epoch 26 : train_loss: 0.039029696154304676 val_loss: 0.009406605528460609 
val_f1: tensor(0.8965) 

t=19.103574752807617s
F1 score tensor(0.9647)
epoch 27 : train_loss: 0.04872878287332477 val_loss: 0.004084518800179164 
val_f1: tensor(0.9433) 

t=19.10965847969055s
F1 score tensor(0.9748)
epoch 28 : train_loss: 0.038144156963414985 val_loss: 0.004025954753160477 
val_f1: tensor(0.9095) 

t=19.144503831863403s
F1 score tensor(0.9803)
epoch 29 : train_loss: 0.03492613803066405 val_loss: 0.00022073276340961456 
val_f1: tensor(0.9451) 

t=19.162553548812866s
F1 score tensor(0.9695)
epoch 30 : train_loss: 0.04098039593570578 val_loss: 0.002522028154797024 
val_f1: tensor(0.9318) 

t=19.20454216003418s
F1 score tensor(0.9785)
epoch 31 : train_loss: 0.036245859340178196 val_loss: 0.0006071250989205308 
val_f1: tensor(0.9433) 

t=19.170579433441162s
F1 score tensor(0.9661)
epoch 32 : train_loss: 0.049284403863431955 val_loss: 0.00294496760600143 
val_f1: tensor(0.9508) 

t=19.14738440513611s
F1 score tensor(0.9782)
epoch 33 : train_loss: 0.036820366932654086 val_loss: 0.002335992538266712 
val_f1: tensor(0.9500) 

t=19.140580892562866s
F1 score tensor(0.9789)
epoch 34 : train_loss: 0.03632871795411187 val_loss: 0.005797412660386827 
val_f1: tensor(0.9307) 

t=19.17143726348877s
F1 score tensor(0.9831)
epoch 35 : train_loss: 0.03269611490476462 val_loss: 0.0018639574862188762 
val_f1: tensor(0.9508) 

t=19.194289922714233s
F1 score tensor(0.9810)
epoch 36 : train_loss: 0.0348962977051413 val_loss: 0.0009755511871642537 
val_f1: tensor(0.9404) 

t=19.18006205558777s
F1 score tensor(0.9809)
epoch 37 : train_loss: 0.030510010470831652 val_loss: 0.0006712728904353248 
val_f1: tensor(0.9491) 

t=19.190290212631226s
F1 score tensor(0.9809)
epoch 38 : train_loss: 0.033684849733325806 val_loss: 0.0012819709049330817 
val_f1: tensor(0.9385) 

t=19.140616178512573s
F1 score tensor(0.9783)
epoch 39 : train_loss: 0.03197238101037564 val_loss: 0.0006697005074885157 
val_f1: tensor(0.9451) 

t=19.131787300109863s
F1 score tensor(0.9795)
epoch 40 : train_loss: 0.03174842608069288 val_loss: 0.001262696563369698 
val_f1: tensor(0.9423) 

t=19.131247520446777s
F1 score tensor(0.9804)
epoch 41 : train_loss: 0.03461431463559469 val_loss: 0.0009317058655950758 
val_f1: tensor(0.9508) 

t=19.124019145965576s
F1 score tensor(0.9773)
epoch 42 : train_loss: 0.03232110573990661 val_loss: 0.00044607810883058444 
val_f1: tensor(0.9364) 

t=19.1271870136261s
F1 score tensor(0.9782)
epoch 43 : train_loss: 0.03140048245341735 val_loss: 0.006891256819168727 
val_f1: tensor(0.9295) 

t=19.14726495742798s
F1 score tensor(0.9817)
epoch 44 : train_loss: 0.028235461844568266 val_loss: 0.005195021215412352 
val_f1: tensor(0.9194) 

t=19.169780492782593s
F1 score tensor(0.9817)
epoch 45 : train_loss: 0.02936192703962234 val_loss: 0.008619111445215013 
val_f1: tensor(0.9237) 

t=19.162467002868652s
F1 score tensor(0.9825)
epoch 46 : train_loss: 0.02855000277567241 val_loss: 0.009364098310470581 
val_f1: tensor(0.9508) 

t=19.193898916244507s
F1 score tensor(0.9809)
epoch 47 : train_loss: 0.031929480574484685 val_loss: 0.0005901965002218882 
val_f1: tensor(0.9404) 

t=19.23000478744507s
F1 score tensor(0.9853)
epoch 48 : train_loss: 0.024914961575564963 val_loss: 0.0005585035930077235 
val_f1: tensor(0.9364) 

t=19.218684434890747s
F1 score tensor(0.9852)
epoch 49 : train_loss: 0.029244690963906455 val_loss: 0.0012166210346751744 
val_f1: tensor(0.9250) 

t=19.202510833740234s
F1 score tensor(0.9812)
epoch 50 : train_loss: 0.028597012022797616 val_loss: 0.00037082163099613454 
val_f1: tensor(0.9637) 

========resnet152========

t=303.00514578819275s
F1 score tensor(0.9403)
epoch 1 : train_loss: 0.085914036328042 val_loss: 0.01304752379655838 
val_f1: tensor(0.9125) 

t=297.2962079048157s
F1 score tensor(0.9473)
epoch 2 : train_loss: 0.08771865426298277 val_loss: 0.006291707356770833 
val_f1: tensor(0.9285) 

t=297.14491844177246s
F1 score tensor(0.9431)
epoch 3 : train_loss: 0.0816395757144984 val_loss: 0.007452222208182017 
val_f1: tensor(0.9262) 

t=297.1380515098572s
F1 score tensor(0.9421)
epoch 4 : train_loss: 0.08195972977275098 val_loss: 0.014101689060529074 
val_f1: tensor(0.9220) 

t=297.238614320755s
F1 score tensor(0.9539)
epoch 5 : train_loss: 0.06781321420207803 val_loss: 0.019213075439135235 
val_f1: tensor(0.9125) 

t=297.2084164619446s
F1 score tensor(0.9469)
epoch 6 : train_loss: 0.07765965909915573 val_loss: 0.0029319427493545744 
val_f1: tensor(0.9385) 

t=297.266086101532s
F1 score tensor(0.9540)
epoch 7 : train_loss: 0.06351969578521854 val_loss: 0.002209357503387663 
val_f1: tensor(0.9085) 

t=297.2454354763031s
F1 score tensor(0.9447)
epoch 8 : train_loss: 0.07793851748660759 val_loss: 0.011493436992168427 
val_f1: tensor(0.9285) 

t=297.1967964172363s
F1 score tensor(0.9548)
epoch 9 : train_loss: 0.06676375956768975 val_loss: 0.010366742809613546 
val_f1: tensor(0.9085) 

t=297.13459181785583s
F1 score tensor(0.9552)
epoch 10 : train_loss: 0.0686182359084027 val_loss: 0.002437533810734749 
val_f1: tensor(0.9262) 

t=297.11710262298584s
F1 score tensor(0.9515)
epoch 11 : train_loss: 0.06805360072901771 val_loss: 0.009803198277950287 
val_f1: tensor(0.9433) 

t=297.1314911842346s
F1 score tensor(0.9668)
epoch 12 : train_loss: 0.053903990850588425 val_loss: 0.007352677484353383 
val_f1: tensor(0.9508) 

t=297.2697253227234s
F1 score tensor(0.9642)
epoch 13 : train_loss: 0.060333619722061686 val_loss: 0.006594918254348967 
val_f1: tensor(0.9250) 

t=297.27229928970337s
F1 score tensor(0.9586)
epoch 14 : train_loss: 0.06048648512956721 val_loss: 0.001581054801742236 
val_f1: tensor(0.9364) 

t=297.2639055252075s
F1 score tensor(0.9662)
epoch 15 : train_loss: 0.058467748390947594 val_loss: 0.013398884071244134 
val_f1: tensor(0.8983) 

t=297.27154636383057s
F1 score tensor(0.9669)
epoch 16 : train_loss: 0.04868750340868661 val_loss: 0.014861534039179483 
val_f1: tensor(0.9375) 

t=297.2680838108063s
F1 score tensor(0.9595)
epoch 17 : train_loss: 0.055664179403978734 val_loss: 0.01491155723730723 
val_f1: tensor(0.9523) 

t=297.264554977417s
F1 score tensor(0.9618)
epoch 18 : train_loss: 0.05402884301786989 val_loss: 0.00153020645181338 
val_f1: tensor(0.9237) 

t=297.2928307056427s
F1 score tensor(0.9643)
epoch 19 : train_loss: 0.05743811532199659 val_loss: 0.005258800668848885 
val_f1: tensor(0.9250) 

t=297.2666416168213s
F1 score tensor(0.9671)
epoch 20 : train_loss: 0.047417441112805665 val_loss: 0.0013731399344073401 
val_f1: tensor(0.9516) 

t=297.2849555015564s
F1 score tensor(0.9680)
epoch 21 : train_loss: 0.0457040913414532 val_loss: 0.006777456237210168 
val_f1: tensor(0.9442) 

t=297.2778582572937s
F1 score tensor(0.9726)
epoch 22 : train_loss: 0.04355165306018827 val_loss: 0.00602149839202563 
val_f1: tensor(0.9508) 

t=297.2967357635498s
F1 score tensor(0.9717)
epoch 23 : train_loss: 0.041387604302324264 val_loss: 0.001803416551815139 
val_f1: tensor(0.9307) 

t=297.2688534259796s
F1 score tensor(0.9724)
epoch 24 : train_loss: 0.04546317976129092 val_loss: 0.0010575225783718957 
val_f1: tensor(0.9451) 

t=297.35152769088745s
F1 score tensor(0.9776)
epoch 25 : train_loss: 0.042283597942671655 val_loss: 0.0005834284238517284 
val_f1: tensor(0.9250) 

t=297.3191497325897s
F1 score tensor(0.9715)
epoch 26 : train_loss: 0.03933464423005963 val_loss: 0.0013047166996532017 
val_f1: tensor(0.9375) 

t=297.31332659721375s
F1 score tensor(0.9649)
epoch 27 : train_loss: 0.049090961526136526 val_loss: 0.0019210409373044968 
val_f1: tensor(0.9307) 

t=297.2918508052826s
F1 score tensor(0.9733)
epoch 28 : train_loss: 0.03644283250388172 val_loss: 0.00830662747224172 
val_f1: tensor(0.9566) 

t=297.3106937408447s
F1 score tensor(0.9747)
epoch 29 : train_loss: 0.03954871310162599 val_loss: 0.01029175685511695 
val_f1: tensor(0.9340) 

t=297.3027687072754s
F1 score tensor(0.9795)
epoch 30 : train_loss: 0.036328609064106406 val_loss: 0.011055479447046915 
val_f1: tensor(0.9385) 

t=297.33516454696655s
F1 score tensor(0.9777)
epoch 31 : train_loss: 0.033477588309504 val_loss: 0.004570778459310532 
val_f1: tensor(0.9508) 

t=297.32631969451904s
F1 score tensor(0.9763)
epoch 32 : train_loss: 0.03314072956664511 val_loss: 0.006102134370141559 
val_f1: tensor(0.9375) 

t=297.29340505599976s
F1 score tensor(0.9761)
epoch 33 : train_loss: 0.03767073328066387 val_loss: 0.0029028813458151287 
val_f1: tensor(0.9580) 

t=297.287535905838s
F1 score tensor(0.9761)
epoch 34 : train_loss: 0.03516774842038429 val_loss: 0.013518892228603363 
val_f1: tensor(0.9573) 

t=297.26994347572327s
F1 score tensor(0.9825)
epoch 35 : train_loss: 0.02558839337233408 val_loss: 0.0002962009360392888 
val_f1: tensor(0.9643) 

t=297.2898807525635s
F1 score tensor(0.9775)
epoch 36 : train_loss: 0.03514046173393266 val_loss: 0.007819352050622305 
val_f1: tensor(0.9274) 

t=297.338326215744s
F1 score tensor(0.9859)
epoch 37 : train_loss: 0.023814233551721697 val_loss: 0.004378300574090745 
val_f1: tensor(0.9508) 

t=297.3154544830322s
F1 score tensor(0.9796)
epoch 38 : train_loss: 0.03317491028078452 val_loss: 0.005901666565073861 
val_f1: tensor(0.9580) 

t=297.2803952693939s
F1 score tensor(0.9812)
epoch 39 : train_loss: 0.031208129480400665 val_loss: 0.011389619774288602 
val_f1: tensor(0.9237) 

t=297.29759764671326s
F1 score tensor(0.9825)
epoch 40 : train_loss: 0.02728185281847362 val_loss: 0.0028569884598255157 
val_f1: tensor(0.9329) 

t=297.2287504673004s
F1 score tensor(0.9803)
epoch 41 : train_loss: 0.026746444202913545 val_loss: 0.012252176801363627 
val_f1: tensor(0.9442) 

t=297.2791085243225s
F1 score tensor(0.9874)
epoch 42 : train_loss: 0.01937206585627095 val_loss: 0.003073398851686054 
val_f1: tensor(0.9442) 

t=297.25949692726135s
F1 score tensor(0.9809)
epoch 43 : train_loss: 0.029331590451392126 val_loss: 0.0020746205829911763 
val_f1: tensor(0.9559) 

t=297.32357907295227s
F1 score tensor(0.9760)
epoch 44 : train_loss: 0.028265907223635343 val_loss: 0.0007682177238166332 
val_f1: tensor(0.9395) 

t=297.1191327571869s
F1 score tensor(0.9860)
epoch 45 : train_loss: 0.026971203628582536 val_loss: 0.00024537043645977974 
val_f1: tensor(0.9508) 

t=297.20972752571106s
F1 score tensor(0.9803)
epoch 46 : train_loss: 0.028629514962193316 val_loss: 0.0005426441008845965 
val_f1: tensor(0.9500) 

t=297.14074540138245s
F1 score tensor(0.9902)
epoch 47 : train_loss: 0.020774172699469844 val_loss: 0.0012664641771051618 
val_f1: tensor(0.9637) 

t=297.1798861026764s
F1 score tensor(0.9853)
epoch 48 : train_loss: 0.02261860327317989 val_loss: 0.00036985352117982175 
val_f1: tensor(0.9631) 

t=297.2135853767395s
F1 score tensor(0.9874)
epoch 49 : train_loss: 0.021125269151166266 val_loss: 0.0001604947287382351 
val_f1: tensor(0.9573) 

t=297.1813995838165s
F1 score tensor(0.9867)
epoch 50 : train_loss: 0.020757816234649517 val_loss: 0.007866421507464515 
val_f1: tensor(0.9643) 

========densnet121========

t=19.16087794303894s
F1 score tensor(0.8963)
epoch 1 : train_loss: 0.14966078093390406 val_loss: 0.02422055933210585 
val_f1: tensor(0.9016) 

t=13.746406555175781s
F1 score tensor(0.8920)
epoch 2 : train_loss: 0.15197222206143685 val_loss: 0.0084493069185151 
val_f1: tensor(0.9274) 

t=13.77516794204712s
F1 score tensor(0.9064)
epoch 3 : train_loss: 0.142879585425059 val_loss: 0.028412676519817777 
val_f1: tensor(0.9016) 

t=13.779662132263184s
F1 score tensor(0.8849)
epoch 4 : train_loss: 0.15122467040279766 val_loss: 0.014367491006851196 
val_f1: tensor(0.8893) 

t=13.813270092010498s
F1 score tensor(0.8953)
epoch 5 : train_loss: 0.1451524427091634 val_loss: 0.01957822839419047 
val_f1: tensor(0.9016) 

t=13.80980658531189s
F1 score tensor(0.8873)
epoch 6 : train_loss: 0.14854604221972417 val_loss: 0.012041151523590088 
val_f1: tensor(0.9274) 

t=13.816355228424072s
F1 score tensor(0.8902)
epoch 7 : train_loss: 0.15345557047813027 val_loss: 0.013058368530538347 
val_f1: tensor(0.9139) 

t=13.802316904067993s
F1 score tensor(0.9057)
epoch 8 : train_loss: 0.13302626286023927 val_loss: 0.013479207124974992 
val_f1: tensor(0.9318) 

t=13.799336433410645s
F1 score tensor(0.9043)
epoch 9 : train_loss: 0.13193303686969074 val_loss: 0.012363226877318488 
val_f1: tensor(0.8911) 

t=13.8148934841156s
F1 score tensor(0.8895)
epoch 10 : train_loss: 0.15135701304232632 val_loss: 0.015311852097511292 
val_f1: tensor(0.9262) 

t=13.839423656463623s
F1 score tensor(0.8940)
epoch 11 : train_loss: 0.14220072296850475 val_loss: 0.009810536271995969 
val_f1: tensor(0.9306) 

t=13.823505640029907s
F1 score tensor(0.9008)
epoch 12 : train_loss: 0.137829435184414 val_loss: 0.008765819172064463 
val_f1: tensor(0.9262) 

t=13.851056337356567s
F1 score tensor(0.9026)
epoch 13 : train_loss: 0.13408089029016318 val_loss: 0.011729828185505338 
val_f1: tensor(0.9254) 

t=13.830801963806152s
F1 score tensor(0.9044)
epoch 14 : train_loss: 0.13452015292865258 val_loss: 0.017900185452567205 
val_f1: tensor(0.9139) 

t=13.837975978851318s
F1 score tensor(0.9060)
epoch 15 : train_loss: 0.1315536624579518 val_loss: 0.011809575061003367 
val_f1: tensor(0.9350) 

t=13.85262942314148s
F1 score tensor(0.9030)
epoch 16 : train_loss: 0.13114010294278464 val_loss: 0.011242022116978964 
val_f1: tensor(0.9307) 

t=13.836900234222412s
F1 score tensor(0.9046)
epoch 17 : train_loss: 0.13023646116072748 val_loss: 0.01912370158566369 
val_f1: tensor(0.9166) 

t=13.862825155258179s
F1 score tensor(0.9124)
epoch 18 : train_loss: 0.12217152932727779 val_loss: 0.008010257449414995 
val_f1: tensor(0.9095) 

t=13.850450992584229s
F1 score tensor(0.9044)
epoch 19 : train_loss: 0.13203849091573996 val_loss: 0.006471285389529334 
val_f1: tensor(0.9207) 

t=13.806489706039429s
F1 score tensor(0.9059)
epoch 20 : train_loss: 0.12510950992136827 val_loss: 0.007301794985930125 
val_f1: tensor(0.9329) 

t=13.806978702545166s
F1 score tensor(0.9149)
epoch 21 : train_loss: 0.12573826260128876 val_loss: 0.012246323128541311 
val_f1: tensor(0.9095) 

t=13.849923849105835s
F1 score tensor(0.9015)
epoch 22 : train_loss: 0.12891338331003985 val_loss: 0.020436209109094407 
val_f1: tensor(0.9207) 

t=13.817626237869263s
F1 score tensor(0.9152)
epoch 23 : train_loss: 0.11709479265558867 val_loss: 0.014748521977000766 
val_f1: tensor(0.9055) 

t=13.81836199760437s
F1 score tensor(0.9074)
epoch 24 : train_loss: 0.12344234393058735 val_loss: 0.012545451521873474 
val_f1: tensor(0.9262) 

t=13.81023359298706s
F1 score tensor(0.9124)
epoch 25 : train_loss: 0.11759711137432376 val_loss: 0.01602437761094835 
val_f1: tensor(0.9318) 

t=13.825177192687988s
F1 score tensor(0.9223)
epoch 26 : train_loss: 0.11388578647632658 val_loss: 0.01532086067729526 
val_f1: tensor(0.9262) 

t=13.844201803207397s
F1 score tensor(0.9256)
epoch 27 : train_loss: 0.1124563287069768 val_loss: 0.007418423891067505 
val_f1: tensor(0.9000) 

t=13.851038455963135s
F1 score tensor(0.9156)
epoch 28 : train_loss: 0.12317412436284401 val_loss: 0.012582180400689444 
val_f1: tensor(0.9262) 

t=13.875795125961304s
F1 score tensor(0.9179)
epoch 29 : train_loss: 0.1105953230764027 val_loss: 0.02004027697775099 
val_f1: tensor(0.8929) 

t=13.851557970046997s
F1 score tensor(0.9187)
epoch 30 : train_loss: 0.12073761956007392 val_loss: 0.005929868254396651 
val_f1: tensor(0.9055) 

t=13.840865135192871s
F1 score tensor(0.9236)
epoch 31 : train_loss: 0.1081826325138042 val_loss: 0.005251908053954442 
val_f1: tensor(0.9003) 

t=13.81281065940857s
F1 score tensor(0.9206)
epoch 32 : train_loss: 0.10763809365438826 val_loss: 0.008246286047829522 
val_f1: tensor(0.9194) 

t=13.827390909194946s
F1 score tensor(0.9229)
epoch 33 : train_loss: 0.10743273603787393 val_loss: 0.013536208205752902 
val_f1: tensor(0.8760) 

t=13.82481074333191s
F1 score tensor(0.9248)
epoch 34 : train_loss: 0.11193755193165054 val_loss: 0.01333472298251258 
val_f1: tensor(0.9181) 

t=13.830970764160156s
F1 score tensor(0.9169)
epoch 35 : train_loss: 0.11258154769462568 val_loss: 0.01192964365084966 
val_f1: tensor(0.9340) 

t=13.822193384170532s
F1 score tensor(0.9206)
epoch 36 : train_loss: 0.11339153199928043 val_loss: 0.00769730160633723 
val_f1: tensor(0.9038) 

t=13.818410634994507s
F1 score tensor(0.9288)
epoch 37 : train_loss: 0.1005726696026546 val_loss: 0.015858037604226008 
val_f1: tensor(0.9395) 

t=13.810212135314941s
F1 score tensor(0.9181)
epoch 38 : train_loss: 0.11568754941317035 val_loss: 0.0076123856835895115 
val_f1: tensor(0.9385) 

t=13.852560758590698s
F1 score tensor(0.9227)
epoch 39 : train_loss: 0.111438091016478 val_loss: 0.011335990495151944 
val_f1: tensor(0.9262) 

t=13.838096380233765s
F1 score tensor(0.9222)
epoch 40 : train_loss: 0.10197409499942521 val_loss: 0.005783632397651672 
val_f1: tensor(0.9442) 

t=13.816253900527954s
F1 score tensor(0.9211)
epoch 41 : train_loss: 0.10918039568688398 val_loss: 0.002299629979663425 
val_f1: tensor(0.9508) 

t=13.81264066696167s
F1 score tensor(0.9228)
epoch 42 : train_loss: 0.11076019899804651 val_loss: 0.010101260410414802 
val_f1: tensor(0.9296) 

t=13.828189849853516s
F1 score tensor(0.9346)
epoch 43 : train_loss: 0.10666360293319564 val_loss: 0.013631899323728349 
val_f1: tensor(0.9340) 

t=13.835029602050781s
F1 score tensor(0.9299)
epoch 44 : train_loss: 0.09881217885808444 val_loss: 0.012510299682617188 
val_f1: tensor(0.9032) 

t=13.828596353530884s
F1 score tensor(0.9276)
epoch 45 : train_loss: 0.10136214121716258 val_loss: 0.007736395630571578 
val_f1: tensor(0.8891) 

t=13.849472999572754s
F1 score tensor(0.9194)
epoch 46 : train_loss: 0.1172459274760367 val_loss: 0.014956124954753451 
val_f1: tensor(0.9262) 

t=13.855374097824097s
F1 score tensor(0.9231)
epoch 47 : train_loss: 0.10649078836043675 val_loss: 0.011334753698772855 
val_f1: tensor(0.9194) 

t=13.853562831878662s
F1 score tensor(0.9345)
epoch 48 : train_loss: 0.09742997287784094 val_loss: 0.011882509622308943 
val_f1: tensor(0.9038) 

t=13.828805208206177s
F1 score tensor(0.9372)
epoch 49 : train_loss: 0.10190920005150048 val_loss: 0.012254113952318827 
val_f1: tensor(0.9375) 

t=13.817769765853882s
F1 score tensor(0.9370)
epoch 50 : train_loss: 0.0987268255126697 val_loss: 0.011633226441012489 
val_f1: tensor(0.9085) 

In [1]:
print("f1_50:\n",f1_50)
print("f1_101:\n",f1_101)
print("f1_152:\n",f1_152)
print("f1_121:\n",f1_121)

NameError: name 'f1_50' is not defined

In [ ]:
# for best model
model=model152
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_152.pth")

In [ ]:
# for best model
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
# for best model
model=model50
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_50.pth")

Train loss 0.5575281381607056 t=4.891497373580933s
Train loss 0.5705251097679138 t=10.128498792648315s


In [ ]:
# for best model
model=model34
optimizer = torch.optim.AdamW(model.parameters(), 0.000005)
# model=model101
epochs = 0
last_acc = 0
val_acc = 0.001
while last_acc < val_acc:
    last_acc = val_acc
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    # train_acc = validate(train_loader, model)
    print(
        "epoch {} :".format(epochs + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
    if val_acc>last_acc:
        torch.save(model.state_dict(),"./model/baseline_34.pth")

In [ ]:
# 对测试集多次预测
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model152
model.load_state_dict(torch.load("./model/baseline_152.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model34
model.load_state_dict(torch.load("./model/baseline_34.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += 0.9*predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit6.csv', index=None)

In [ ]:
model.train()
train_loss = 0.0
model = model.to(device)
for i, (input, target) in enumerate(train_loader):
    input = input.to(device)
    target = target.to(device)
    output = model(input)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
epochs=4
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_test.pth")



In [ ]:
epochs=3
model101= XunFeiNet().to(device)
model=model101
optimizer = torch.optim.AdamW(model.parameters(), 0.0001)
for epoch in range(epochs):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc,val_loss = validate(val_loader, model, criterion)
    print(
        "epoch {} :".format(epoch + 1), "train_loss:", train_loss, 'val_loss:', val_loss,"val_f1:", val_acc
    )
    epochs += 1
torch.save(model.state_dict(),"./model/baseline_101.pth")

In [ ]:
pred = None
model=model101
model.load_state_dict(torch.load("./model/baseline_test.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model.load_state_dict(torch.load("./model/baseline_101.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
model=model50
model.load_state_dict(torch.load("./model/baseline_50.pth"))
for _ in range(3):
    if pred is None:
        pred = predict(test_loader, model)
    else:
        pred += predict(test_loader, model)
    print(_+1)
submit = pd.DataFrame(
    {
        'name': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

# 生成提交结果
submit = submit.sort_values(by='name')
submit.to_csv('submit5.csv', index=None)

In [ ]:
model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
model.fc

In [ ]:
val_acc = validate(val_loader, model)
val_acc

In [ ]:
dataset=XunFeiDataset(train_path[:-1000], train_label[:-1000],
            A.Compose([
            # A.RandomRotate90(),
            A.Resize(256, 256),
            A.RandomCrop(224, 224),
            # A.HorizontalFlip(p=0.5),
            # A.RandomBrightnessContrast(p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        )
train_dataset, test_dataset=torch.utils.data.random_split(dataset=dataset,lengths=[0.9,0.1],generator=torch.Generator().manual_seed(42))
train_dataset

In [ ]:
model.load_state_dict(torch.load("./model/baseline_test.pth"))
val_acc,val_loss = validate(val_loader, model, criterion)
print(val_loss)
len(val_loader.dataset)